In [ ]:
pip install google-cloud-translate google-cloud-texttospeech moviepy


In [3]:
import os
from google.cloud import translate_v2 as translate
from google.cloud import texttospeech
from moviepy import VideoFileClip, AudioFileClip, concatenate_videoclips
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:


# Set up Google Cloud credentials
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path_to_your_service_account_key.json'

# Initialize Google Cloud clients
translate_client = translate.Client()
tts_client = texttospeech.TextToSpeechClient()

def translate_text(text, target_language):
    result = translate_client.translate(text, target_language=target_language)
    return result['translatedText']

def text_to_speech(text, language_code):
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code=language_code, ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
    response = tts_client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)
    return response.audio_content

def dub_video(video_path, target_language, output_path):
    video = VideoFileClip(video_path)
    original_audio = video.audio
    translated_audio_clips = []

    for i, clip in enumerate(video.iter_frames()):
        text = extract_text_from_audio_clip(original_audio.subclip(i, i+1))
        translated_text = translate_text(text, target_language)
        translated_audio = text_to_speech(translated_text, target_language)
        translated_audio_clip = AudioFileClip(translated_audio)
        translated_audio_clips.append(translated_audio_clip)

    final_audio = concatenate_videoclips(translated_audio_clips)
    final_video = video.set_audio(final_audio)
    final_video.write_videofile(output_path, codec='libx264')

def extract_text_from_audio_clip(audio_clip):
    # Implement your own method to extract text from audio clip
    return "Extracted text"

# Example usage
dub_video('input_video.mp4', 'es', 'output_video.mp4')
